# Загрузка Pandas и очистка данных

In [158]:
import pandas as pd

In [159]:
df = pd.read_csv('main_task.csv')

In [160]:
#  Коды по очистке данных и генерации новых признаков

In [161]:
df.sample(3)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
28372,id_2435,London,"['Japanese', 'Sushi', 'Asian', 'Vegetarian Fri...",2440,4.0,$$ - $$$,311.0,"[['Delicious sushi place, but don’t go to the....",/Restaurant_Review-g186338-d3383114-Reviews-Do...,d3383114
34052,id_821,Bratislava,['Eastern European'],822,2.5,$$ - $$$,15.0,"[['Just terrible!', 'Good'], ['10/05/2017', '1...",/Restaurant_Review-g274924-d2052591-Reviews-Le...,d2052591
23108,id_507,Oslo,"['American', 'Pizza', 'Vegetarian Friendly', '...",508,3.5,$$ - $$$,275.0,"[['Pizza buffet is a good choice', 'Great pizz...",/Restaurant_Review-g190479-d3384365-Reviews-Pe...,d3384365


In [162]:
# Переименуем столбцы
# df.rename(columns={'Restaurant_id': 'restaurant_id','City': 'city','Cuisine Style': 'cuisine_style','Ranking': 'ranking', 'Rating': 'rating','Price Range': 'price_range','Number of Reviews': 'nor','Reviews': 'reviews','URL_TA': 'url_ta','ID_TA': 'id_ta'}, inplace=True)
df.columns=['restaurant_id','city','cuisine_style','ranking','rating','price_range','nor','reviews','url_ta','id_ta']

In [163]:
df.sample(3)

,restaurant_id,city,cuisine_style,ranking,rating,price_range,nor,reviews,url_ta,id_ta
10902,id_13353,Paris,NaN,13355,2.5,NaN,7.0,"[['Fresh, petite'], ['01/03/2014']]",/Restaurant_Review-g187147-d6223165-Reviews-Ca...,d6223165
5188,id_2318,Hamburg,['German'],2322,3.5,NaN,2.0,"[['Schlemmer-Sommer Excellence', 'Long waits, ...",/Restaurant_Review-g187331-d4355928-Reviews-N_...,d4355928
2453,id_414,Warsaw,"['Bar', 'Pub', 'Gastropub']",415,4.5,$$ - $$$,77.0,"[['Great craft beer!', 'Good Brewery in Warsaw...",/Restaurant_Review-g274856-d10290442-Reviews-B...,d10290442


In [164]:
# смотрим на количество уникальных значений по столбцу, 
# определяем сколько ресторанов относятся к среднему ценовому сегменту
print(df['price_range'].value_counts())

$$ - $$$    18412
$            6279
$$$$         1423
Name: price_range, dtype: int64


In [165]:
# определяем сколько ресторанов относятся к среднему ценовому сегменту
df[df.price_range =='$$ - $$$'].restaurant_id.count()

18412

In [166]:
# посчитаем сколько городов в наборе данных
df['city'].nunique()

31

In [167]:
# Сколько типов кухонь представлено в наборе данных?
def make_list(n_str):
    n_str = n_str.replace('[','')
    n_str = n_str.replace(']','')
    n_list = n_str.split(', ')
    return list(n_list)

df['cuisine_style'] = df['cuisine_style'].fillna('none')
df['cuisine_style'] = df['cuisine_style'].apply(make_list)

cs_list = []

for i in range(0, len(df['cuisine_style'])):
    
    one_list = df['cuisine_style'][i] 
    
    for elem in one_list:             
        if elem not in cs_list and elem != 'none':
            cs_list.append(elem)
            
print(len(cs_list))

125


In [168]:
cs_liste = []

for i in range (0, len(df['cuisine_style'])):
    one_list = list(df['cuisine_style'][i])
    for elem in one_list:
        if elem != 'none':
            cs_liste.append(elem)

import collections
c = collections.Counter()

for word in cs_liste:
    c[word] += 1
    
print(c.most_common(1))

[("'Vegetarian Friendly'", 11189)]


In [169]:
def cnt_kitchen(n_list):
    cnt = 0
    for elem in n_list:
        if elem != 'none':
            cnt += 1
        else:
            cnt = 1
    return cnt

df['cnt_kitchen'] = df['cuisine_style'].apply(cnt_kitchen)

print(df['cnt_kitchen'].mean())

2.6224


In [170]:
print(df['cnt_kitchen'])

0        3
1        1
2        7
3        1
4        3
        ..
39995    4
39996    5
39997    2
39998    5
39999    1
Name: cnt_kitchen, Length: 40000, dtype: int64


In [171]:
# определяем когда был оставлен самый свежий отзыв
import re
date_comment = df['reviews'][0]

def search_date(n_str):
    found_date = re.findall('\d{2}\/\d{2}\/\d{4}', n_str)
    if len(found_date) == 0:
        found_date = 'no_date'
    return found_date

df['reviews_date'] = df['reviews'].apply(search_date)

len_rev = len(df['reviews_date'])

cs_list = []

for i in range (0, len_rev):
    one_list = df['reviews_date'][i]
    for elem in one_list:
        if elem not in 'no_date':
            cs_list.append(pd.to_datetime(elem, format='%m/%d/%Y'))

# pd.to_datetime(df['date_time_column'], format='%m/%d/%Y')

print(min(cs_list))
print(max(cs_list))

2004-04-21 00:00:00
2018-02-26 00:00:00


In [172]:
# Какое максимальное количество дней отделяет даты публикации отзывов, размещённых на сайте ресторана?
cs_list = []

for i in range (0, len_rev):
    one_list = df['reviews_date'][i]
    if len(one_list) == 2:
        f_date = pd.to_datetime(one_list[0], format='%m/%d/%Y')
        s_date = pd.to_datetime(one_list[1], format='%m/%d/%Y')
        cs_list.append(f_date - s_date)

print(max(cs_list))

3207 days 00:00:00


In [173]:
# создаём дополнительный признак об уровне цен
def range_value(n_str):
    if n_str == 0:
        res = 0
    elif n_str == '$':
        res = 1
    elif n_str == '$$ - $$$':
        res = 2
    else:
        res = 3
    return res

df['price_range'] = df['price_range'].fillna(0)
df['price_range'] = df['price_range'].apply(range_value)

print(df['price_range'])

0        2
1        0
2        3
3        0
4        2
        ..
39995    2
39996    2
39997    0
39998    2
39999    2
Name: price_range, Length: 40000, dtype: int64


In [175]:
# df = df[['price_range','rating']]
df

,restaurant_id,city,cuisine_style,ranking,rating,price_range,nor,reviews,url_ta,id_ta,cnt_kitchen,reviews_date
0,id_5569,Paris,"['European', 'French', 'International']",5570,3.5,2,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,3,"[12/31/2017, 11/20/2017]"
1,id_1535,Stockholm,[none],1537,4.0,0,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,1,"[07/06/2017, 06/19/2016]"
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353,4.5,3,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,7,"[01/08/2018, 01/06/2018]"
3,id_3456,Berlin,[none],3458,5.0,0,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,1,no_date
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621,4.0,2,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,3,"[11/18/2017, 02/19/2017]"
...,...,...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500,4.5,2,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414,4,"[12/16/2017, 11/12/2017]"
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341,3.5,2,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036,5,"[12/21/2017, 12/12/2017]"
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652,4.5,0,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615,2,"[11/03/2016, 04/12/2008]"
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641,4.0,2,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838,5,"[07/11/2017, 06/18/2017]"


In [182]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df[['price_range','cnt_kitchen','ranking']]
y = df['rating']

In [183]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [184]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [185]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [186]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [187]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.49789321090992333
